# Multi-Agent System Demo with LangGraph

This notebook demonstrates a sophisticated multi-agent system using LangGraph's Swarm architecture. The system includes:
- **Main Agent**: Coordination and task orchestration
- **Alice**: Math calculations specialist
- **Bob**: Weather specialist with pirate personality

## Features
- Memory checkpointer for state inspection
- Tool integration with interrupts
- Agent handoff capabilities
- Real-time state tracking


## Example Scenario
We'll implement the Portuguese example: "Qual a temperatura atual no Campeche? Agora pegue essa temperatura e multiplique pelo numero que eu estou pensando e me retorne o final."
(What's the current temperature in Campeche? Now take that temperature and multiply it by the number I'm thinking of and return the result.)


## 1. Imports and Configuration

First, we'll import all necessary libraries and set up our language models.


In [42]:
# Core imports for type checking and agents
from typing_extensions import Literal, TypedDict
from typing import Annotated
import operator
import os

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, AIMessage, ToolMessage, HumanMessage, BaseMessage
from langchain_core.runnables import RunnableConfig
from langchain.tools import tool
from langchain_core.tools import tool, InjectedToolCallId, BaseTool
from langchain.chat_models import init_chat_model
from langchain_groq import ChatGroq

# LangGraph imports
from langgraph.types import Command, interrupt
from langgraph.graph import StateGraph, MessagesState
from langgraph.constants import START, END
from langgraph.prebuilt import create_react_agent
from langgraph.prebuilt.chat_agent_executor import AgentState, AgentStateWithStructuredResponse
from langgraph.checkpoint.memory import MemorySaver

# LangGraph Swarm imports
from langgraph_swarm import (
    create_handoff_tool,
    create_swarm,
    SwarmState,
    add_active_agent_router,
)

# Pydantic for data models
from pydantic import BaseModel


In [44]:
# Initialize language models
model = init_chat_model("openai:gpt-4o-mini", temperature=0)
model_groq = model

print("✅ All imports loaded successfully!")
print(f"🤖 Model initialized: {model}")

MAIN_AGENT_PROMPT = """You are the Main Coordination Agent responsible for task orchestration and completion.

CORE RESPONSIBILITIES:
1. Analyze incoming tasks and develop a strategic execution plan
2. Gather necessary information from users when requirements are unclear
3. Delegate specialized tasks to appropriate expert agents
4. Coordinate between agents to ensure seamless task completion

WORKFLOW PROCESS:
1. ANALYZE: Break down the user's request and identify required expertise
2. PLAN: Structure a clear strategy outlining steps and agent assignments
3. GATHER: Use `ask_user` tool to collect missing information before proceeding
4. DELEGATE: Hand off specific, well-defined subtasks to specialized agents
5. COORDINATE: Monitor progress and facilitate inter-agent communication

HANDOFF PROTOCOL:
- Always provide clear, specific instructions when transferring tasks
- Include relevant context and expected deliverables
- Ensure each agent receives the exact information needed for their specialization

Remember: Strategic planning before action ensures optimal task completion."""


✅ All imports loaded successfully!
🤖 Model initialized: client=<openai.resources.chat.completions.completions.Completions object at 0x10f819c70> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10f81bb00> root_client=<openai.OpenAI object at 0x10f819970> root_async_client=<openai.AsyncOpenAI object at 0x10f819280> model_name='gpt-4o-mini' temperature=0.0 model_kwargs={} openai_api_key=SecretStr('**********')


## 2. Tool Definitions

We define three core tools for our multi-agent system:
- **get_weather**: Retrieves weather information for a location
- **calculate_math**: Performs mathematical calculations
- **ask_user**: Enables human-in-the-loop interactions with interrupts


In [ ]:
def get_weather(location: str, tool_call_id: Annotated[str, InjectedToolCallId]):
    """
    Get the weather for a given location.
    Returns temperature data and updates the system state.
    """
    print(f"🌤️ Getting weather for {location}")
    import random

    temperature = random.randint(20, 30)

    return Command(
        update={
            "location": location,
            "temperature": f"{temperature} degrees",  # Simulated temperature for Campeche
            "date": f"2025-01-{temperature}",
            "time": f"12:{temperature}:00",
            "tool_call_id": tool_call_id,
            "messages": [
                ToolMessage(
                    f"The weather for {location} is {temperature} degrees.",
                    tool_call_id=tool_call_id,
                )
            ],
        }
    )


def calculate_math(expression: str, tool_call_id: Annotated[str, InjectedToolCallId]):
    """
    Calculate a simple math expression.
    Supports: addition (+), multiplication (*), subtraction (-), division (/)
    Example: "2 + 3" or "70 * 3"
    """
    print(f"🔢 Calculating math for {expression}")

    try:
        if "+" in expression:
            parts = expression.split("+")
            result = sum(float(part.strip()) for part in parts)
        elif "*" in expression:
            parts = expression.split("*")
            result = 1
            for part in parts:
                result *= float(part.strip())
        elif "-" in expression:
            parts = expression.split("-")
            result = float(parts[0].strip()) - float(parts[1].strip())
        elif "/" in expression:
            parts = expression.split("/")
            result = float(parts[0].strip()) / float(parts[1].strip())
        else:
            result = float(expression.strip())

        return Command(
            update={
                "math_expression": expression,
                "math_result": result,
                "messages": [
                    ToolMessage(
                        f"The result of {expression} is {result}",
                        tool_call_id=tool_call_id,
                    )
                ],
            }
        )
    except Exception as e:
        error_msg = f"Could not calculate {expression}. Please use format like '2 + 3' or '10 * 5'"
        print(f"❌ Math error: {e}")
        return Command(
            update={
                "messages": [ToolMessage(error_msg, tool_call_id=tool_call_id)],
            }
        )


def ask_user(question_to_user: str, tool_call_id: Annotated[str, InjectedToolCallId]):
    """
    This tool is used to ask the user any question.
    It's important to always ask for things to make sure you're using the right information.
    Uses interrupt mechanism for human-in-the-loop interaction.
    """
    print(f"❓ Asking user: {question_to_user}")

    user_response = interrupt(
        {
            "type": "question",
            "question": question_to_user,
            "tool_call_id": tool_call_id,
        }
    )

    print(f"👤 User response: {user_response}")
    return f"The user answered with: {user_response}"


print("✅ Tool definitions completed!")
print("🔧 Available tools: get_weather, calculate_math, ask_user")

✅ Tool definitions completed!
🔧 Available tools: get_weather, calculate_math, ask_user


## 3. State Definitions and Workflow Compilation

We define the state schemas that will track information across agent interactions, including temperature, location, math expressions, and results. The memory checkpointer enables state inspection and recovery.


In [46]:
def compile_workflow(workflow: StateGraph):
    """
    Compile the workflow with memory checkpointer for state inspection.
    Enables state recovery and debugging capabilities.
    """
    is_langgraph_api = (
        os.environ.get("LANGGRAPH_API", "false").lower() == "true"
        or os.environ.get("LANGGRAPH_API_DIR") is not None
    )

    if is_langgraph_api:
        print("🌐 Compiling for LangGraph API")
        return workflow.compile()
    else:
        print("💾 Compiling with Memory Checkpointer")
        memory = MemorySaver()
        return workflow.compile(checkpointer=memory)


# Hook functions for debugging and monitoring
def pre_hook_supervisor_node(state, config: RunnableConfig):
    """Pre-execution hook for debugging"""
    print(f"🔍 Pre-hook supervisor: {state}")
    return state


def post_hook_supervisor_node(state, config: RunnableConfig):
    """Post-execution hook for debugging"""
    print(f"✅ Post-hook supervisor: {state}")
    return state

print("✅ Workflow compilation functions ready!")
print("💾 Memory checkpointer will be used for state tracking")


✅ Workflow compilation functions ready!
💾 Memory checkpointer will be used for state tracking


## 4. Multi-Agent System Creation

This section creates the complete multi-agent system with:
- **Main Agent**: Coordinates tasks and manages workflow
- **Alice**: Mathematics specialist 
- **Bob**: Weather specialist with pirate personality

Each agent has specific tools and capabilities, with handoff mechanisms for seamless collaboration.


In [59]:
from sample_agent.utils import create_handoff_tool_with_state_propagation

def strip_tool_messages_node(state: dict) -> Command:
    return Command(update={
        "messages": [
            m for m in state["messages"]
            if m.__class__.__name__ != "ToolMessage"
        ]
    })


def create_multi_agent_system_swarm_mode():
    """Create the multi-agent system with supervisor"""

    # State definitions for tracking data across agents
    class FullState(AgentState):
        temperature: float
        location: str
        weather: str
        math_expression: str
        math_result: str

    class AliceState(AgentStateWithStructuredResponse):
        math_expression: str
        math_result: str

    class AliceOutput(BaseModel):
        math_expression: str
        math_result: str

    class BobState(AgentStateWithStructuredResponse):
        location: str
        weather: str
        temperature: float

    class BobOutput(BaseModel):
        location: str
        weather: str
        temperature: float

    # Create Main Agent - Coordination and Planning
    print("🤖 Creating Main Agent...")
    main_agent_model = init_chat_model("openai:gpt-4o-mini", temperature=0)
    main_agent_tools = [
        ask_user,
        create_handoff_tool(
            agent_name="Alice",
            description="Transfer to Alice, she can help with any math",
        ),
        create_handoff_tool(
            agent_name="Bob", 
            description="Transfer to Bob, he can help with weather"
        ),
    ]
    main_agent_model_bind_tools = main_agent_model.bind_tools(
        main_agent_tools,
        parallel_tool_calls=False,
    )

    main_agent = create_react_agent(
        main_agent_model_bind_tools,
        main_agent_tools,
        prompt=MAIN_AGENT_PROMPT,
        name="main_agent",
        state_schema=FullState,
    )

    # Create Alice - Math Specialist
    print("🔢 Creating Alice (Math Specialist)...")
    alice_model = init_chat_model("openai:gpt-4o-mini", temperature=0)
    alice_tools = [
        calculate_math,
        create_handoff_tool(
            agent_name="Bob", 
            description="Transfer to Bob, he can help with weather"
        ),
        create_handoff_tool(
            agent_name="main_agent",
            description="Use this tool to send or ask the user for information to complete the task.",
        ),
    ]
    alice_model_bind_tools = alice_model.bind_tools(
        alice_tools,
        parallel_tool_calls=False,
    )

    alice = create_react_agent(
        alice_model_bind_tools,
        alice_tools,
        prompt="You are Alice, a calculator expert. You are given a math expression and you need to calculate the result. If you need to ask the user for information, handoff to the main_agent.",
        name="Alice",
        state_schema=AliceState,
        response_format=AliceOutput
    )

    # Create Bob - Weather Specialist
    print("🌤️ Creating Bob (Weather Specialist)...")
    bob_model = init_chat_model("openai:gpt-4o-mini", temperature=0)
    bob_tools = [
        ask_user,
        get_weather,
        create_handoff_tool_with_state_propagation(
            agent_name="Alice",
            description="Transfer to Alice, she can help with any math or any calculation",
            propagate_keys=["location", "weather", "temperature"],
        ),
        create_handoff_tool(
            agent_name="main_agent",
            description="Use this tool to send or ask the user for information to complete the task.",
        ),
    ]
    bob_model_bind_tools = bob_model.bind_tools(
        bob_tools,
        parallel_tool_calls=False,
    )

    bob = create_react_agent(
        bob_model_bind_tools,
        bob_tools,
        prompt="You are Bob, you speak like a pirate and you are a weather specialist. You are given a location and you need to return the weather for that location using the `get_weather` tool. If you need any user information, handoff back to the main_agent. To get the number that the user is thinking handoff back to the main_agent.",
        name="Bob",
        state_schema=BobState,
        response_format=BobOutput,
    )

    # Create Swarm State that combines SwarmState with our custom state
    class FullSwarmState(SwarmState, FullState):
        temperature: float
        location: str
        weather: str
        math_expression: str
        math_result: str

    # Build the workflow graph
    print("🕸️ Building workflow graph...")
    workflow = (
        StateGraph(FullSwarmState)
        .add_node(
            main_agent,
            destinations=(
                "Alice",
                "Bob",
            ),
        )
        .add_node(alice, destinations=("main_agent", "Bob"))
        .add_node(bob, destinations=("main_agent", "Alice"))
        
    )
    workflow = workflow.add_node("cleanup_messages", strip_tool_messages_node)
    workflow = workflow.add_edge("main_agent", "cleanup_messages")
    workflow = workflow.add_edge("Alice", "cleanup_messages")
    workflow = workflow.add_edge("Bob", "cleanup_messages")
    workflow = workflow.add_edge("cleanup_messages", END)
    
    # Add the router that enables tracking of the last active agent
    workflow = add_active_agent_router(
        builder=workflow,
        route_to=["main_agent", "Alice", "Bob"],
        default_active_agent="main_agent",
    )

    # Compile the workflow with memory checkpointer
    print("🔧 Compiling workflow...")
    graph = compile_workflow(workflow)
    
    print("✅ Multi-agent system created successfully!")
    print("🎯 Agents: Main Agent, Alice (Math), Bob (Weather)")
    print("💾 Memory checkpointer enabled for state inspection")
    
    return graph


# Create the multi-agent system
print("🚀 Initializing Multi-Agent System...")
graph = create_multi_agent_system_swarm_mode()
print("🎉 System ready for execution!")


🚀 Initializing Multi-Agent System...
🤖 Creating Main Agent...
🔢 Creating Alice (Math Specialist)...
🌤️ Creating Bob (Weather Specialist)...
🕸️ Building workflow graph...
🔧 Compiling workflow...
💾 Compiling with Memory Checkpointer
✅ Multi-agent system created successfully!
🎯 Agents: Main Agent, Alice (Math), Bob (Weather)
💾 Memory checkpointer enabled for state inspection
🎉 System ready for execution!


## 5. Example Execution

Now we'll run the specific example

This demonstrates:
1. Weather query delegation to Bob
2. User interaction for the mystery number
3. Math calculation by Alice
4. Complete state tracking through memory checkpointer


In [60]:
# Execution and State Inspection Functions

def execute_campeche_example(query, thread_id):

    # Create a unique thread for this execution
    thread_config = {"configurable": {"thread_id": thread_id}}
    
    print("🇧🇷 Starting Example...")
    print(f"📝 Query: '{query}'")
    print("=" * 80)
    
    # Initial message
    messages = [
        HumanMessage(content=query)
    ]
    
    # Step-by-step execution with state tracking
    step_count = 0
    max_steps = 10  # Prevent infinite loops
    
    try:
        for step in graph.stream(
            {"messages": messages}, 
            thread_config, 
            stream_mode="values"
        ):
            step_count += 1
            if step_count > max_steps:
                print(f"⚠️ Maximum steps ({max_steps}) reached. Stopping execution.")
                break
                
            print(f"\n📍 Step {step_count}:")
            
            # Show current state
            if hasattr(step, 'get'):
                current_agent = step.get('active_agent', 'Unknown')
                location = step.get('location', 'Not set')
                temperature = step.get('temperature', 'Not set')
                math_expression = step.get('math_expression', 'Not set')
                math_result = step.get('math_result', 'Not set')
                
                print(f"🤖 Active Agent: {current_agent}")
                print(f"📍 Location: {location}")
                print(f"🌡️ Temperature: {temperature}")
                print(f"🔢 Math Expression: {math_expression}")
                print(f"🎯 Math Result: {math_result}")
            
            # Show latest message
            if 'messages' in step and step['messages']:
                latest_message = step['messages'][-1]
                print(f"💬 Latest Message: {latest_message.content[:100]}...")
            
            print("-" * 40)
    
    except Exception as e:
        print(f"❌ Execution error: {e}")
        print("🔧 This is expected if interrupts are triggered for user input")
    
    return thread_config


def inspect_state(thread_config):
    """Inspect the current state using the memory checkpointer"""
    
    print("\n🔍 STATE INSPECTION")
    print("=" * 50)
    
    try:
        # Get the current state from checkpointer
        current_state = graph.get_state(thread_config)
        
        if current_state:
            print("📊 Current State Values:")
            values = current_state.values
            
            for key, value in values.items():
                if key == 'messages':
                    print(f"  💬 {key}: {len(value)} messages")
                    # Show last 2 messages
                    for i, msg in enumerate(value[-2:], start=len(value)-1):
                        msg_type = type(msg).__name__
                        content = str(msg.content)[:80] + "..." if len(str(msg.content)) > 80 else str(msg.content)
                        print(f"    [{i}] {msg_type}: {content}")
                else:
                    print(f"  🎯 {key}: {value}")
            
            print(f"\n📍 Next Steps: {current_state.next}")
            print(f"🆔 State ID: {current_state.config.get('configurable', {}).get('thread_id', 'Unknown')}")
            
        else:
            print("❌ No state found")
            
    except Exception as e:
        print(f"❌ State inspection error: {e}")


def simulate_user_input(thread_config, user_response="3"):
    """Simulate user input to continue execution"""
    
    print(f"\n👤 Simulating user input: '{user_response}'")
    print("=" * 50)
    
    try:
        # Continue execution with user input
        for step in graph.stream(
            None,  # No new input, just continue
            thread_config,
            stream_mode="values"
        ):
            print("📍 Continuing execution after user input...")
            
            # Show current state
            if hasattr(step, 'get'):
                current_agent = step.get('active_agent', 'Unknown')
                location = step.get('location', 'Not set')
                temperature = step.get('temperature', 'Not set')
                math_expression = step.get('math_expression', 'Not set')
                math_result = step.get('math_result', 'Not set')
                
                print(f"🤖 Active Agent: {current_agent}")
                print(f"📍 Location: {location}")
                print(f"🌡️ Temperature: {temperature}")
                print(f"🔢 Math Expression: {math_expression}")
                print(f"🎯 Math Result: {math_result}")
            
            break  # Only show first step
    
    except Exception as e:
        print(f"❌ Continuation error: {e}")


def analyze_final_results(thread_config):
    """Analyze the final results of the Campeche example"""
    
    print("\n🎯 FINAL ANALYSIS")
    print("=" * 50)
    
    try:
        final_state = graph.get_state(thread_config)
        
        if final_state and final_state.values:
            values = final_state.values
            
            location = values.get('location', 'Unknown')
            temperature = values.get('temperature', 'Unknown')
            math_expression = values.get('math_expression', 'Unknown')
            math_result = values.get('math_result', 'Unknown')
            
            print(f"📍 Location Queried: {location}")
            print(f"🌡️ Temperature Retrieved: {temperature}")
            print(f"🔢 Math Expression Calculated: {math_expression}")
            print(f"🎯 Final Result: {math_result}")
            
            # Show final messages
            messages = values.get('messages', [])
            if messages:
                print(f"\n💬 Total Messages Exchanged: {len(messages)}")
                print("\n📜 Message History (last 3):")
                for i, msg in enumerate(messages[-3:], start=len(messages)-2):
                    msg_type = type(msg).__name__
                    content = str(msg.content)[:100] + "..." if len(str(msg.content)) > 100 else str(msg.content)
                    print(f"  [{i}] {msg_type}: {content}")
            
        else:
            print("❌ No final state available")
            
    except Exception as e:
        print(f"❌ Analysis error: {e}")

print("✅ Execution and analysis functions ready!")


✅ Execution and analysis functions ready!


## 6. Run the Demo

Execute the example and observe the multi-agent collaboration in real-time. The system will:

1. **Main Agent**: Analyze the complex request
2. **Bob**: Get weather for Campeche (returns 70°)
3. **Main Agent**: Ask user for the mystery number
4. **Alice**: Calculate 70 × user_number
5. **Final Result**: Display the complete calculation

**Note**: The execution may pause for user input simulation. This demonstrates the interrupt mechanism.


In [61]:
# Execute the Campeche example
thread_config = execute_campeche_example("Qual a temperatura atual no Campeche? Agora pegue essa temperatura e multiplique pelo dobro da temperatura e me retorne o final.", "campeche_demo_002")

# Inspect the current state
inspect_state(thread_config)


🇧🇷 Starting Example...
📝 Query: 'Qual a temperatura atual no Campeche? Agora pegue essa temperatura e multiplique pelo dobro da temperatura e me retorne o final.'

📍 Step 1:
🤖 Active Agent: Unknown
📍 Location: Not set
🌡️ Temperature: Not set
🔢 Math Expression: Not set
🎯 Math Result: Not set
💬 Latest Message: Qual a temperatura atual no Campeche? Agora pegue essa temperatura e multiplique pelo dobro da tempe...
----------------------------------------

📍 Step 2:
🤖 Active Agent: Bob
📍 Location: Not set
🌡️ Temperature: Not set
🔢 Math Expression: Not set
🎯 Math Result: Not set
💬 Latest Message: Successfully transferred to Bob...
----------------------------------------
🌤️ Getting weather for Campeche

📍 Step 3:
🤖 Active Agent: Alice
📍 Location: Campeche
🌡️ Temperature: 26 degrees
🔢 Math Expression: Not set
🎯 Math Result: Not set
💬 Latest Message: Successfully transferred to Alice...
----------------------------------------
🔢 Calculating math for 26 * (2 * 26)
❌ Math error: could not conver

In [62]:
final_state = graph.get_state(thread_config)

In [64]:
final_state.values


{'messages': [HumanMessage(content='Qual a temperatura atual no Campeche? Agora pegue essa temperatura e multiplique pelo dobro da temperatura e me retorne o final.', additional_kwargs={}, response_metadata={}, id='a65f4c28-8f32-4b96-ba0e-6007f12aa48f'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ex9IaEu6H7mfXNOR1APUps7V', 'function': {'arguments': '{}', 'name': 'transfer_to_bob'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 342, 'total_tokens': 354, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BrSrAKrRHC4RHpJpdJiPGXw5PgTYs', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='main_agent', id='run--165f35

In [ ]:
graph.

## 8. Additional Utilities and Debugging

These utility functions help with debugging and state management of the multi-agent system.


In [67]:
def reset_system():
    """Reset the system state and create a new graph instance"""
    global graph
    print("🔄 Resetting multi-agent system...")
    graph = create_multi_agent_system_swarm_mode()
    print("✅ System reset complete!")
    return graph


def debug_checkpointer_state(thread_config):
    """Debug the checkpointer state in detail"""
    
    print("\n🔧 DETAILED CHECKPOINTER DEBUG")
    print("=" * 60)
    
    try:
        state = graph.get_state(thread_config)
        
        if state:
            print(f"📊 State Config: {state.config}")
            print(f"📍 Next Steps: {state.next}")
            print(f"🔄 Tasks: {getattr(state, 'tasks', 'Not available')}")
            
            print("\n📋 All State Values:")
            for key, value in state.values.items():
                if key == 'messages':
                    print(f"  💬 {key}: {len(value)} total messages")
                    for i, msg in enumerate(value):
                        msg_type = type(msg).__name__
                        content = str(msg.content)[:50] + "..." if len(str(msg.content)) > 50 else str(msg.content)
                        print(f"    [{i}] {msg_type}: {content}")
                else:
                    print(f"  🎯 {key}: {value}")
        
        else:
            print("❌ No state found for the given thread configuration")
            
    except Exception as e:
        print(f"❌ Debug error: {e}")


def test_individual_tools():
    """Test each tool individually"""
    
    print("\n🧪 INDIVIDUAL TOOL TESTING")
    print("=" * 50)
    
    print("\n1. Testing get_weather tool:")
    try:
        weather_result = get_weather("Campeche", "test_tool_call_1")
        print(f"   Result: {weather_result}")
    except Exception as e:
        print(f"   Error: {e}")
    
    print("\n2. Testing calculate_math tool:")
    try:
        math_result = calculate_math("70 * 3", "test_tool_call_2")
        print(f"   Result: {math_result}")
    except Exception as e:
        print(f"   Error: {e}")
    
    print("\n3. Testing ask_user tool:")
    try:
        print("   Note: ask_user tool requires interrupt mechanism, skipping direct test")
    except Exception as e:
        print(f"   Error: {e}")


def show_system_info():
    """Display system information and configuration"""
    
    print("\n📋 SYSTEM INFORMATION")
    print("=" * 50)
    
    print(f"🤖 Language Model: {model}")
    print(f"🔧 LangGraph API Mode: {os.environ.get('LANGGRAPH_API', 'false')}")
    print(f"📁 LangGraph API Dir: {os.environ.get('LANGGRAPH_API_DIR', 'Not set')}")
    
    # Check if graph has checkpointer
    if hasattr(graph, 'checkpointer'):
        print(f"💾 Checkpointer: {type(graph.checkpointer).__name__}")
    else:
        print("💾 Checkpointer: Not available")
    
    # Show graph structure
    if hasattr(graph, 'nodes'):
        print(f"🕸️ Graph Nodes: {list(graph.nodes.keys()) if graph.nodes else 'Not available'}")
    
    
print("✅ Utility functions loaded!")
print("🔧 Use show_system_info() to see available functions")


✅ Utility functions loaded!
🔧 Use show_system_info() to see available functions


In [68]:
# Show system information and available functions
show_system_info()

# Test individual tools
test_individual_tools()



📋 SYSTEM INFORMATION
🤖 Language Model: client=<openai.resources.chat.completions.completions.Completions object at 0x10f819c70> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10f81bb00> root_client=<openai.OpenAI object at 0x10f819970> root_async_client=<openai.AsyncOpenAI object at 0x10f819280> model_name='gpt-4o-mini' temperature=0.0 model_kwargs={} openai_api_key=SecretStr('**********')
🔧 LangGraph API Mode: false
📁 LangGraph API Dir: Not set
💾 Checkpointer: InMemorySaver
🕸️ Graph Nodes: ['__start__', 'main_agent', 'Alice', 'Bob', 'cleanup_messages']

🧪 INDIVIDUAL TOOL TESTING

1. Testing get_weather tool:
🌤️ Getting weather for Campeche
   Result: Command(update={'location': 'Campeche', 'temperature': '20 degrees', 'date': '2025-01-20', 'time': '12:20:00', 'tool_call_id': 'test_tool_call_1', 'messages': [ToolMessage(content='The weather for Campeche is 20 degrees.', tool_call_id='test_tool_call_1')]})

2. Testing calculate_math tool:
🔢 Cal

## 9. Summary and Conclusion

This notebook demonstrates a sophisticated multi-agent system using LangGraph that successfully handles complex, multi-step tasks requiring coordination between specialized agents.

### Key Features Demonstrated:

1. **🤖 Multi-Agent Coordination**: Main agent orchestrates task distribution
2. **🔧 Specialized Agents**: 
   - Alice (mathematics)
   - Bob (weather with personality)
3. **💾 Memory Checkpointer**: Complete state tracking and inspection
4. **🔄 Agent Handoffs**: Seamless task delegation
5. **👤 Human-in-the-Loop**: Interactive user input with interrupts
6. **📊 State Management**: Real-time monitoring and debugging

### Expected Workflow:
1. User asks for Campeche temperature and multiplication
2. Main Agent analyzes and plans execution
3. Bob retrieves weather information (70°)
4. Main Agent requests the mystery number from user
5. Alice calculates the final result (70 × user_number)
6. System provides complete calculation result

### Memory Checkpointer Benefits:
- **State Recovery**: Resume execution after interrupts
- **Debugging**: Inspect state at any point
- **Audit Trail**: Complete message and state history
- **Error Recovery**: Rollback and retry capabilities

### Next Steps:
- Experiment with different queries
- Add more specialized agents
- Implement more complex calculations
- Enhance error handling and validation

The system successfully demonstrates enterprise-grade multi-agent coordination with comprehensive state management and debugging capabilities.
